# Retriever 성능 및 동작 테스트

이 노트북은 분리된 `BM25Retriever`와 `VectorRetriever`의 동작을 검증하고, 각 검색 방식의 특성을 비교하기 위해 작성되었습니다.

In [ ]:
# 환경 설정
import sys
import os
from dotenv import load_dotenv

# 프로젝트 루트 경로 설정
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

load_dotenv(os.path.join(project_root, ".env"))
print(f"Project Root: {project_root}")

In [ ]:
import asyncio
from src.retrieval.bm25_retriever import BM25Retriever
from src.retrieval.vector_retriever import VectorRetriever

# 리트리버 초기화
bm25 = BM25Retriever(version="v3", collection_name="care_guides")
vector = VectorRetriever(version="v3", collection_name="care_guides")
print("✅ Retrievers initialized.")

## 1. 단순 키워드 검색 (BM25 vs Vector)

In [ ]:
async def compare_search(query, specialist=None, filters=None):
    print(f"\n🧐 Query: '{query}' (Specialist: {specialist}, Filters: {filters})")
    
    print("\n--- [BM25 Results] ---")
    bm25_res = await bm25.search(query, specialist=specialist, filters=filters, limit=3)
    for i, r in enumerate(bm25_res):
        print(f"{i+1}. [{r.get('name_ko', 'Unknown')}] {r.get('text', '')[:50]}...")
        
    print("\n--- [Vector Results] ---")
    vec_res = await vector.search(query, specialist=specialist, filters=filters, limit=3)
    for i, r in enumerate(vec_res):
        print(f"{i+1}. [{r.get('name_ko', 'Unknown')}] {r.get('text', '')[:50]}...")

# Case 1: 명확한 키워드 (특정 품종)
await compare_search("메인쿤")

## 2. 추상적/의미론적 검색 (Vector 유리)

In [ ]:
# Case 2: 특징/성격 묘사
await compare_search("활동적이고 개냥이 같은 친구 추천해줘")

## 3. 필터링 및 전문가 모드 테스트

In [ ]:
# Case 3: 전문가 필터 (Matchmaker)
await compare_search("털 안빠지는 고양이", specialist="Matchmaker", filters={"categories": "Breeds"})

## 4. 미등록 품종 테스트 (Unknown Breed)

In [ ]:
# Case 4: 없는 품종 (갤럭시 캣)
# BM25는 결과가 없어야 하고, Vector는 유사한 것을 가져올 것임.
await compare_search("갤럭시 캣")